In [1]:
import pandas as pd
import numpy as np 
from keras.layers import Dense , SimpleRNN , GRU , LSTM , Embedding
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical

In [2]:
amazon = pd.read_table(r"D:\data set\amazonreviews.tsv")

In [3]:
amazon.head()

,label,review
0,pos,Stuning even for the non-gamer: This sound tra...
1,pos,The best soundtrack ever to anything.: I'm rea...
2,pos,Amazing!: This soundtrack is my favorite music...
3,pos,Excellent Soundtrack: I truly like this soundt...
4,pos,"Remember, Pull Your Jaw Off The Floor After He..."


In [4]:
amazon.label.value_counts()

neg    5097
pos    4903
Name: label, dtype: int64

In [5]:
amazon.replace( { "label"  : { "pos" : 1 , "neg" :0}} , inplace = True)

In [6]:
amazon_x = amazon.iloc[ : ,1]
amazon_y = amazon.iloc[ : ,0]

In [7]:
from sklearn.model_selection import train_test_split

train_x , test_x , train_y , test_y = train_test_split( amazon_x , amazon_y , test_size= 0.2) 

In [8]:
train_y  = to_categorical( train_y)
test_y   = to_categorical(test_y)

In [9]:
max_num_words = 20000    ## How many words do we consider from the entire CORPUS
seq_len = 100  ## How many tokens ( words to consider the each documnet / messages)
embedding_size = 200  ## vector lengths of each word

In [10]:
from keras.preprocessing.text import Tokenizer  # is used to tokinization ( it gives number to each and every words. )
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam

In [11]:
tokenizer = Tokenizer(num_words = max_num_words)
tokenizer.fit_on_texts(amazon.review)
train_x = tokenizer.texts_to_sequences(train_x) # will convert the text to sequences of IDs
train_x = pad_sequences(train_x , maxlen = seq_len)

In [12]:
test_x  =  tokenizer.texts_to_sequences(test_x)
test_x  =  pad_sequences(test_x , maxlen = seq_len)
model = Sequential()  # iniitialize the network
model.add(Embedding(input_dim = max_num_words,    
                   input_length = seq_len,
                    output_dim = embedding_size))

In [13]:
model.add(LSTM(5)) 
model.add(Dense(2 , activation  = 'softmax'))  # this 2 bcoz u have 2 level in target variable



adam = Adam(lr = .001)


model.compile(optimizer= 'adam' , loss = 'categorical_crossentropy' , metrics = ['accuracy'])

C:\Users\Priti\anaconda3\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [14]:
model.fit(train_x , train_y , epochs = 10 , batch_size= 32, validation_split= .2)

Epoch 1/10
200/200 [==============================] - 15s 65ms/step - loss: 0.5802 - accuracy: 0.6997 - val_loss: 0.5159 - val_accuracy: 0.7738
Epoch 2/10
200/200 [==============================] - 13s 64ms/step - loss: 0.3432 - accuracy: 0.8669 - val_loss: 0.4532 - val_accuracy: 0.7994
Epoch 3/10
200/200 [==============================] - 13s 64ms/step - loss: 0.1909 - accuracy: 0.9389 - val_loss: 0.4953 - val_accuracy: 0.7981
Epoch 4/10
200/200 [==============================] - 13s 63ms/step - loss: 0.1236 - accuracy: 0.9628 - val_loss: 0.5518 - val_accuracy: 0.7987
Epoch 5/10
200/200 [==============================] - 13s 63ms/step - loss: 0.0842 - accuracy: 0.9742 - val_loss: 0.5820 - val_accuracy: 0.7937
Epoch 6/10
200/200 [==============================] - 13s 63ms/step - loss: 0.0517 - accuracy: 0.9853 - val_loss: 0.6144 - val_accuracy: 0.8094
Epoch 7/10
200/200 [==============================] - 13s 63ms/step - loss: 0.0272 - accuracy: 0.9947 - val_loss: 0.6865 - val_accuracy:

In [15]:
pred_test = model.predict(test_x)
pred_test

array([[0.99432003, 0.0056799 ],
       [0.8923386 , 0.10766147],
       [0.89292806, 0.10707197],
       ...,
       [0.9952192 , 0.00478081],
       [0.00378111, 0.99621886],
       [0.99890304, 0.00109698]], dtype=float32)

In [16]:
pred_test_c=np.argmax(pred_test,axis=1)
pred_test_c

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [17]:
test_y_c=np.argmax(test_y,axis=1)
test_y_c

array([0, 0, 0, ..., 0, 1, 0], dtype=int64)

In [18]:
from sklearn.metrics import *

In [19]:
tab=confusion_matrix(test_y_c,pred_test_c)
tab

array([[917, 127],
       [251, 705]], dtype=int64)

In [20]:
acc=accuracy_score(test_y_c,pred_test_c)
acc*100

81.10000000000001

In [21]:
recall_score(test_y_c,pred_test_c)

0.7374476987447699

In [22]:
precision_score(test_y_c,pred_test_c)

0.8473557692307693